In [7]:
import requests
import json
import getpass

## Connect to the musixmatch API to get lyrics:
You can sign up for a free developer account and generate a personal API key

https://developer.musixmatch.com/ 

In [8]:
api_key = getpass.getpass("api_key")

Defining some helper functions to collect lyrics

In [15]:
def get_songs(artist):
    api_url = "http://api.musixmatch.com/ws/1.1/track.search&apikey=" + api_key 
    url = api_url + "&q_artist=" + str(artist)
    x = requests.get(url)
    response = json.loads(x.text)

    return response['message']["body"]["track_list"]

In [16]:
# len("\n...\n\n******* This Lyrics is NOT for Commercial use *******") = 59

def get_lyrics(track_id):
    api_url = "http://api.musixmatch.com/ws/1.1/track.lyrics.get?apikey=" + api_key
    url = api_url + "&track_id="  + str(track_id)
    x = requests.get(url)
    response = json.loads(x.text)
    return response['message']["body"]["lyrics"]["lyrics_body"][:-59]

In [17]:
songs = get_songs("hozier")

song_list = []
for song in songs:
    mysong = {}
    mysong["id"] = song["track"]["track_id"]
    mysong["name"] = song["track"]["track_name"].split("(")[0].split("-")[0]
    mysong["artist"] = song["track"]["artist_name"]
    if (song["track"]["has_lyrics"] == 1):
        mysong["lyrics"] = get_lyrics(mysong["id"])
    else:
        mysong["lyrics"] = None
    song_list.append(mysong)

In [18]:
with open('hozier_songs.json', 'w') as f:
    json.dump(song_list, f)

## Pydantic Model example for validating data input

In [4]:
import json
with open('hozier_songs.json', 'r') as f:
  songs = json.load(f)

In [13]:
song = songs[0]
song

{'id': 84741219,
 'name': 'Cherry Wine ',
 'artist': 'Hozier',
 'lyrics': "Her eyes and words are so icy\nOh but she burns\nLike rum on the fire\nHot and fast and angry as she can be\nI walk my days on a wire.\nIt looks ugly, but it's clean,\nOh momma, don't fuss over me.\nThe way she tells me I'm hers and she is mine\nOpen hand or closed fist would be fine\nThe blood is rare and sweet as cherry wine.\nCalls of guilty thrown at me"}

In [8]:
from pydantic import BaseModel

class Song(BaseModel):
    name: str
    artist: str
    lyrics: str
    id : int

    model_config = {
        "json_schema_extra": {
            "examples": [
                {
                    "name": "Take me to church",
                    "artist": "Hozier",
                    "lyrics": "Her eyes and words are so icy\nOh but she burns\nLike rum on the fire",
                    "id": 0,
                }
            ]
        }
    }
    
example = Song.model_validate(song)

In [9]:
example

Song(name='Cherry Wine ', artist='Hozier', lyrics="Her eyes and words are so icy\nOh but she burns\nLike rum on the fire\nHot and fast and angry as she can be\nI walk my days on a wire.\nIt looks ugly, but it's clean,\nOh momma, don't fuss over me.\nThe way she tells me I'm hers and she is mine\nOpen hand or closed fist would be fine\nThe blood is rare and sweet as cherry wine.\nCalls of guilty thrown at me", id=84741219)

In [10]:
example.artist

'Hozier'